# Queens Problem

### importem llibreries que utilitzarem i definim variables globals


In [94]:
import numpy as np
import random

INITIAL_POPULATION = 100
BOARD_SIZE = 10
MUTATION_CHANCE = 20

### Definir Agent, població i display


In [95]:
def init_agent(board_size):
    agent= list(range(1,board_size+1))
    random.shuffle(agent)
    return agent
            

def display_agent(agent):
    array_zeroes = np.zeros((len(agent),len(agent)))
    count=0
    for x in agent:
        array_zeroes[count][x-1]=1
        count+=1
    return array_zeroes

def init_population(initial_population_size, board_size):
    population=[]
    for x in range(0, initial_population_size):
        population.append(init_agent(board_size))
        
    return population



display_agent(init_agent(BOARD_SIZE))


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

### definir crossover


In [96]:
def crossover(parent1, parent2):
    leng= len(parent1)+1
    half=int(leng/2)
    soon=parent1[0:half]
    for x in parent2:
        if x not in soon : soon.append(x)
    soon2=parent2[0:half]
    for x in parent1:
        if x not in soon2 : soon2.append(x)
    return soon, soon2

### definir mutació


In [97]:
def mutation(agent, board_size):
    pos1= random.randint(0,board_size-1)
    pos2= random.randint(0,board_size-1)
    agent[pos2], agent[pos1] = agent[pos1], agent[pos2]
    return agent
    

def mutation_with_chance(agent, mutation_chance):
    chance= random.randint(1,100)
    if chance <= mutation_chance:
        agent=mutation(agent, BOARD_SIZE)
    return agent


### definir crossover de tota la població amb mutació


In [98]:

def replicate(population, mutation_chance):
    soons=[]
    
    for x in range(0, len(population),2):
        try:
            soons.extend(crossover(population[x], population[x+1]))
        except:
            soons.extend(population[x])
    for agent in soons:
        mutation_with_chance(agent, mutation_chance)
    
    return soons
'''
def replicate(population, mutation_chance):
    aux=population.copy()
    soons=[]
    
    while len(aux) != 0:
        par1=random.randint(0,len(aux)-1)
        par2=random.randint(0,len(aux)-2)
        try:
            soons.extend(crossover(population[par1], population[par2]))
            aux.remove(par1)
            aux.remove(par2)
        except:
            soons.extend(population[par1])
            aux.remove(par1)
    for agent in soons:
        mutation_with_chance(agent, mutation_chance)
    
    return soons
'''

### definir fitness function

mirar diagonals i treure repetides


In [99]:
def fitness_agent(agent):
    display=display_agent(agent)
    fitness=0

    for i in range(len(display)):
        for j in range(len(display[i])):
            if display[i][j] == 1:  # Encontramos un 1
                # Verificamos las diagonales y sumamos al contador global
                fitness += verificar_diagonales(display, i, j)

    return fitness
        


# Función para verificar las diagonales
def verificar_diagonales(matriz, fila, col):
    n = len(matriz)
    # Inicializamos el contador para esta posición
    contador_local = 0
    # Verificamos la diagonal superior izquierda
    i, j = fila - 1, col - 1
    while i >= 0 and j >= 0:
        if matriz[i][j] == 1:
            contador_local += 1
        i -= 1
        j -= 1

    # Verificamos la diagonal inferior derecha
    i, j = fila + 1, col + 1
    while i < n and j < n:
        if matriz[i][j] == 1:
            contador_local += 1
        i += 1
        j += 1

    # Verificamos la diagonal superior derecha
    i, j = fila - 1, col + 1
    while i >= 0 and j < n:
        if matriz[i][j] == 1:
            contador_local += 1
        i -= 1
        j += 1

    # Verificamos la diagonal inferior izquierda
    i, j = fila + 1, col - 1
    while i < n and j >= 0:
        if matriz[i][j] == 1:
            contador_local += 1
        i += 1
        j -= 1

    return contador_local
    

def fitness_population(population):
    fitness=[]
    for agent in population:
        fitness.append(fitness_agent(agent))
    return fitness



### Population Selection (by score)


In [100]:
def sort_population(population, fitness_list):
    aux=zip(population, fitness_list)
    aux=sorted(aux, key=lambda x: x[1])
    population,fitness_list=zip(*aux)
    return population

def select_population(population, fitness_list, population_size, pie_vector):
    population=sort_population(population, fitness_list)
    new_population=[]
    #Primeros
    new_population.extend(population[:int(pie_vector[0]*population_size)])
    #Ultimos
    new_population.extend(population[-int(pie_vector[1]*population_size):])
    #Mitad
    aux=population[:int(pie_vector[0]*population_size)]
    new_population.extend(aux[:int((pie_vector[2])*population_size)])
    return new_population



### definir main

(seguir els pasos de classe)


In [101]:
import time


def main(initial_population, board_size, pie_vector, mutation_change):
    pre = time.time()
    _population = init_population(initial_population, board_size)
    print(time.time()-pre)
    _agent = _population[0]
    best_fitness=10
    
    gen=0

    while best_fitness!=0:
        soons=replicate(_population, mutation_change)
        _population.extend(soons)
        
        fitness_list=fitness_population(_population)
        
        
        _population=select_population(_population, fitness_list, initial_population, pie_vector)
        _population=list(_population)
        best_fitness=fitness_list[0]
        
        gen+=1
        
        if gen%10==0:
            print("current gen: ",gen, "best fitness: ", fitness_list[0])
    print(_agent)
    print(gen)

        
    return _population[0]
 
# main(INITIAL_POPULATION, BOARD_SIZE, [0.6,0.1,0.3], 10)

### Provar el codi


In [102]:
POPULATION_SIZE = 80
BOARD_SIZE = 100
MUTATION_CHANCE = 25
pie_vector=[0.7,0.2,0,1]

random.seed(21)
agent = main(POPULATION_SIZE, BOARD_SIZE, pie_vector, MUTATION_CHANCE)
display_agent(agent)

0.0018312931060791016


ValueError: list.remove(x): x not in list